In [ ]:
!pip install transformers torch torchvision pillow opencv-python pandas scikit-learn tqdm

import os
import cv2
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from PIL import Image, ImageSequence
import requests
from io import BytesIO
from transformers import BlipProcessor, BlipModel, DistilBertTokenizer, DistilBertModel,BlipForConditionalGeneration
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, accuracy_score

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on:", device)


Running on: cuda


In [ ]:
# Uploading the csv file
from google.colab import files
uploaded = files.upload()

csv_file_name = list(uploaded.keys())[0]
data = pd.read_csv(csv_file_name)


Saving behaviour_simulation_train.xlsx - Sheet1 (2).csv to behaviour_simulation_train.xlsx - Sheet1 (2).csv


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
#Loading Models

blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base", torch_dtype=torch.float16).to("cuda")
blip_model.eval()

bert_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
bert_model = DistilBertModel.from_pretrained("distilbert-base-uncased").to(device)
bert_model.eval()

device = "cuda" if torch.cuda.is_available() else "cpu"
blip_model.to(device)
bert_model.to(device)



Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): DistilBertSdpaAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

In [ ]:
from datetime import datetime
def load_image_from_url(url):
    """Load image safely from URL"""
    try:
        return Image.open(requests.get(url, stream=True, timeout=8).raw).convert("RGB")
    except:
        return None

def get_image_embedding(url):
    """Extract BLIP image embedding"""
    try:
        image = load_image_from_url(url)
        if image is None:
            return np.zeros(768)
        inputs = blip_processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = blip_model.vision_model(**inputs)
            emb = outputs.last_hidden_state.mean(dim=1)
        return emb.cpu().numpy().flatten()
    except:
        return np.zeros(768)

def get_text_embedding(text):
    """Extract BERT text embedding"""
    text = str(text).replace("<mention>", "").replace("<hyperlink>", "").strip()
    inputs = bert_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128).to(device)
    with torch.no_grad():
        outputs = bert_model(**inputs)
        emb = outputs.last_hidden_state.mean(dim=1)
    return emb.cpu().numpy().flatten()

def extract_metadata_features(row):
    """Extract numeric metadata features"""
    followers = row.get('followers_count', 0)
    timestamp = row.get('timestamp', None)

    hour, day_of_week, month = 0, 0, 0
    if pd.notna(timestamp):
        try:
            dt = datetime.strptime(str(timestamp), "%Y-%m-%d %H:%M:%S")
            hour = dt.hour
            day_of_week = dt.weekday()
            month = dt.month
        except:
            pass

    return np.array([followers, hour, day_of_week, month], dtype=float)

In [ ]:
data["image_embedding"] = data["media"].apply(get_image_embedding)
data["text_embedding"] = data["content"].apply(get_text_embedding)
data["metadata_features"] = data.apply(extract_metadata_features, axis=1)

In [ ]:
def combine_features(row):
    img_emb = row["image_embedding"]
    txt_emb = row["text_embedding"]
    meta_emb = row["metadata_features"]
    return np.concatenate([img_emb, txt_emb, meta_emb])


In [ ]:
# Create feature matrix X (all combined embeddings)
X = np.vstack(data.apply(combine_features, axis=1).values)
y = data["likes"].values  # target variable

print("Feature Matrix Shape:", X.shape)
print("Target Shape:", y.shape)

# ===================================================
# STEP 6 – Train-test split + Regressor training
# ===================================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

regressor = RandomForestRegressor(
    n_estimators=200, n_jobs=-1, random_state=42
)
regressor.fit(X_train, y_train)

Feature Matrix Shape: (3000, 1540)
Target Shape: (3000,)


KeyboardInterrupt: 

In [ ]:
# Predict on test set
    y_pred = model.predict(X_test)

    # Compute metrics
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
